In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import random

In [ ]:
data_file = pd.read_table('ml-1m/ratings.dat', sep = '::', header=None, engine='python')
movie_file = pd.read_table('ml-1m/movies.dat', sep = '::', header=None, engine='python')

In [ ]:
users = np.unique(data_file[0])
movies = np.unique(movie_file[0])

n_users = len(users)
n_movies = len(movies) 

movie_indices, user_indices = {}, {}
 
for i in range(len(movies)):
    movie_indices[movies[i]] = i 
    
for i in range(len(users)):
    user_indices[users[i]] = i 

In [ ]:
V = sp.lil_matrix((n_users, n_movies))
T = sp.lil_matrix((n_users, n_movies))
for line in data_file.values:
    u, i , r , time = map(int,line)
    V[user_indices[u], movie_indices[i]] = r
    T[user_indices[u], movie_indices[i]] = time

In [ ]:
print(V.shape, T.shape)

In [ ]:
ratio = 0.3
training = []
test = []
for user in range(V.shape[0]):
    nonzero = np.nonzero(V[user,:])[1]
    testsize = len(nonzero) * ratio
    while len(nonzero) > testsize:
        x = random.randint(0, len(nonzero) - 1)
        training.append((user + 1, movies[nonzero[x]], V[user,nonzero[x]], T[user,nonzero[x]]))
        nonzero = np.delete(nonzero, x)
    for movie in nonzero:
        test.append((user + 1, movies[movie], V[user,movie], T[user,movie]))
print(len(training), len(test))

In [ ]:
training_file = open('data/training.csv', 'r+')
test_file = open('data/test.csv', 'r+')
training_file.truncate()
test_file.truncate()
for row in training:
    training_file.write(str(row[0]) + "," + str(row[1]) + "," + str(int(row[2])) + "," + str(int(row[3])) + "\n")
training_file.close()
for row in test:
    test_file.write(str(row[0]) + "," + str(row[1]) + "," + str(int(row[2])) + "," + str(int(row[3])) + "\n")
test_file.close()